# Project management

In [10]:
from os import path, getenv
from mlrun import new_project

project_name = '-'.join(filter(None, ['properties-management']))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/igztraining/mlrun/ModuleIII/conf
Project name: properties-management


In [11]:
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath('jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/igztraining/mlrun/ModuleIII/jobs
MLRun DB path: http://mlrun-api:8080


# Add function to project

In [12]:
project.set_function('/User/igztraining/mlrun/modules/function.yaml',name='properties-handler')

In [13]:
project.functions

[{'url': '/User/igztraining/mlrun/modules/function.yaml',
  'name': 'properties-handler'}]

# Run function as part of a pipeline

In [14]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [19]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io

@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())


def kfpipeline(
    source_file :str = ""
) -> None:
    
    save_properties = funcs['properties-handler'].as_step(handler='save_properties',
                            params={"source_file" : source_file,
                                    "target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            outputs=['condo_properties'])
    
    
    filter_properties = funcs['properties-handler'].as_step(handler='filter_properties',
                                   params={"city" : "SACRAMENTO",
                                           "target_dataset" : "sacramento_condos" },
                                   inputs={"source_data":save_properties.outputs["condo_properties"]})

Overwriting /User/igztraining/mlrun/ModuleIII/conf/workflow.py


In [20]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [21]:
project.save()

In [22]:
arguments = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv"}
run_id = project.run(
    'main',
    arguments=arguments, 
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}')), 
    dirty=True)

> 2020-11-17 19:36:52,718 [info] Pipeline run id=a731b668-d9f5-49bd-a0fd-5913d02f845b, check UI or DB for progress
